In [1]:
from plyfile import PlyData
import matplotlib.gridspec as gridspec
import matplotlib
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg
import random

In [10]:

def readPly(plydata):
    dotList = list(plydata.elements[0])
    for i in range(len(dotList)):
        dotList[i] = [dotList[i][0],dotList[i][1],dotList[i][2]]
    faceList = list(plydata.elements[1])
    for i in range(len(faceList)):
        faceList[i] = [faceList[i][0][0],faceList[i][0][1],faceList[i][0][2]]
    return dotList,faceList
plydata = PlyData.read('sa4_color5.ply')
dotList, faceList = readPly(plydata)

In [5]:

plydata.elements[1]
# plydata.write('123.ply')

PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'int'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar')), count=35462, comments=[])

In [11]:
def generate_vertex(faceList, dotList):
    vertexList = []
    for face in faceList:
        a = dotList[face[0]]
        b = dotList[face[1]]
        c = dotList[face[2]]
        vertexA = [b[0] - a[0], b[1] - a[1], b[2] - a[2]]
        vertexB = [c[0] - a[0], c[1] - a[1], c[2] - a[2]]
        i = vertexA[1] * vertexB[2] - vertexA[2] * vertexB[1]
        j = vertexA[2] * vertexB[0] - vertexA[0] * vertexB[2]
        k = vertexA[0] * vertexB[1] - vertexA[1] * vertexB[0]
        res = math.sqrt(i**2+j**2+k**2)
        i /= res
        j /= res
        k /= res
        vertexList.append([i,j,k])
    return vertexList

def dis(vertexList, i, j):
    a = vertexList[i]
    b = vertexList[j]
    return abs(a[0]*b[0]+a[1]*b[1]*a[2]*b[2])

def makeW(gamma, dotList,faceList,vertexList):
    graph = np.zeros((len(faceList),len(faceList))).astype(np.float32)
    hashMap = {}
    dot_size = len(dotList)
    for i in range(len(faceList)):
        face = faceList[i]
        a = face[0]
        b = face[1]
        c = face[2]
        t = 0
        if(a > b):
            t = a
            a = b
            b = t
        if(a > c):
            t = a
            a = c
            c = t
        if(b > c):
            t = b
            b = c
            c = t
        for j in range(3):
            temp = -1
            if (j == 0):
                temp = a * dot_size + b
            elif (j == 1):
                temp = a * dot_size + c
            elif (j == 2):
                temp = b * dot_size + c
            if (hashMap.get(temp) == None):
                hashMap[temp] = i
            else:
                face_index = hashMap.get(temp)
                distance = dis
                graph[i][face_index] = distance 
                graph[face_index][i] = distance
    return graph

vertexList = generate_vertex(faceList, dotList)
W = makeW(1, dotList, faceList, vertexList)
del faceList
del dotList
del vertexList

MemoryError: Unable to allocate 4.68 GiB for an array with shape (35462, 35462) and data type float32

In [3]:
W = np.zeros((10,10))
W[0:5,0:5] = np.ones((5,5))
W[5:10,5:10] = np.ones((5,5))
W

array([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])

In [14]:
W[W<math.cos(math.pi*80/180)] = 0
# scipy.sparse.csgraph.connected_components(sp.sparse.coo_matrix(W))
W

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [4]:
def getD(W):
    """
    获得度矩阵
    :param W: 邻接矩阵
    :return: D
    """
    D = np.diag(sum(W))
    return D


def getL(W):
    """
    获得拉普拉斯矩阵
    :param W: 邻接矩阵
    :param D: 度矩阵
    :return: L
    """
    #L = D - W
    D = np.diag(np.sum(W, axis=1))
    return D-W


def getEigen(L, cluster_num):
    """
    获得拉普拉斯矩阵的特征矩阵
    :param L:
    :param cluter_num: 聚类数目
    :return:
    """
#     L = sp.sparse.coo_matrix(L)
# # #     M = np.eye(10)
#     eigval, eigvec = sp.sparse.linalg.eigs(A=L,sigma = 0.000000000000001)
#     print(eigval.real)
#     return eigvec.real
  
    eigval, eigvec=np.linalg.eig(L)
    print(1)
    dim=len(eigval)
    print(2)
    #查找前k小的eigval
    dictEigval=dict(zip(eigval,range(0,dim)))
    print(3)
    kEig=np.sort(eigval)[0:cluster_num]
    print(4)
    ix=[dictEigval[k] for k in kEig]
    print(5)
    return eigvec[:,ix]

def cluster(W, cluster_num):
#     D = getD(W)
    L = getL(W)
    eigvec = getEigen(L, cluster_num)
    clf = KMeans(n_clusters=cluster_num)
    s = clf.fit(eigvec)  # 聚类
    label = s.labels_
    return  label

label = cluster(W, 2)
label

MemoryError: Unable to allocate 18.7 GiB for an array with shape (35462, 35462) and data type complex128

In [44]:
help(sp.sparse.linalg.eigs)

Help on function eigs in module scipy.sparse.linalg.eigen.arpack.arpack:

eigs(A, k=6, M=None, sigma=None, which='LM', v0=None, ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, OPpart=None)
    Find k eigenvalues and eigenvectors of the square matrix A.
    
    Solves ``A * x[i] = w[i] * x[i]``, the standard eigenvalue problem
    for w[i] eigenvalues with corresponding eigenvectors x[i].
    
    If M is specified, solves ``A * x[i] = w[i] * M * x[i]``, the
    generalized eigenvalue problem for w[i] eigenvalues
    with corresponding eigenvectors x[i]
    
    Parameters
    ----------
    A : ndarray, sparse matrix or LinearOperator
        An array, sparse matrix, or LinearOperator representing
        the operation ``A * x``, where A is a real or complex square matrix.
    k : int, optional
        The number of eigenvalues and eigenvectors desired.
        `k` must be smaller than N-1. It is not possible to compute all
        eigenvectors of a mat

In [ ]:
colorList = []
for i in range(6):
    colorList.append((random.randint(0,255),random.randint(0,255),random.randint(0,255)))
for i in range(len(label)):
    plydata.elements[1][i]['red'] = colorList[label[i]][0]
    plydata.elements[1][i]['green'] = colorList[label[i]][1]    
    plydata.elements[1][i]['blue'] = colorList[label[i]][2]    
plydata.write('1.ply')

In [53]:
import random
random.randint(0,255)

231

In [24]:
import scipy as sp
import scipy.sparse.linalg
import numpy as np
import time
A = sp.sparse.lil_matrix((1000, 1000))
A[0,:1000]=np.random.rand(1000)
A.setdiag(np.random.rand(10000))
timeCheckin=time.clock()
vals, vecs = sp.sparse.linalg.eigs(A, k=3)
print("first 6 eigenvaluse cost %s" % (time.clock()-timeCheckin))
print (vals)


C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


first 6 eigenvaluse cost 0.7843808000000081
[0.99990442+0.j 0.9992245 +0.j 0.99908282+0.j]


C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
a = [1,2,3,4,4,6]


In [27]:
data1 = pd.DataFrame(list(plydata.elements[0]['x']),columns={'x'})
data2 = pd.DataFrame(list(plydata.elements[0]['y']),columns={'y'})
data3 = pd.DataFrame(list(plydata.elements[0]['z']),columns={'z'})
data = pd.concat([data1,data2],axis=1)
data = pd.concat([data,data3],axis=1)
data.head()

,x,y,z
0,89.477730,10.516972,120.252052
1,89.477730,10.284519,120.610497
2,89.757812,10.284519,120.252052
3,88.277733,11.042942,120.852051
4,88.277733,10.884519,121.011864


In [59]:
from pyecharts.faker import Faker
scatter = Scatter3D(init_opts = opts.InitOpts(width='900px',height='600px')).add("",list(data[1:3].values)    )
scatter.render_notebook()

In [1]:
import open3d as o3d

In [4]:
pkl = o3d.io.read_point_cloud("mmpData/pottery1.ply")
print(pkl)

o3d.visualization.draw_geometries([pkl])

PointCloud with 732684 points.


In [8]:
help(pkl

AttributeError: 'open3d.cpu.pybind.geometry.PointCloud' object has no attribute 'x'